http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/

In [1]:
import numpy as np
import lightgbm as lgb

In [2]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else: return 0

def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


In [3]:
NUM_DIGITS = 10
NUM_DATA = 10
X = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DATA)])
y = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DATA)])

In [4]:
X.shape

(923, 10)

In [5]:
X[0], y[0]

(array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0]), 0)

In [6]:
X_train = X[100:]
y_train = y[100:]
X_valid = X[:100]
y_valid = y[:100]

In [7]:
import collections
c_train = collections.Counter(list(y_train))
c_valid = collections.Counter(list(y_valid))
c_train, c_valid

(Counter({0: 439, 1: 220, 2: 109, 3: 55}),
 Counter({0: 54, 1: 26, 2: 13, 3: 7}))

# Normal

In [8]:
lgbm_params = {
    'boosting_type' : 'gbdt',
    'objective': 'multiclass',
    'num_class': 4,
    'learning_rate': 0.05,
    'min_child_samples': 5,  # Minimum number of data need in a child(min_data_in_leaf)
    'subsample': 0.9,  # Subsample ratio of the training instance.
}

def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df, weight=None)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

In [9]:
model = lgbm_train(X_train, X_valid, y_train, y_valid, lgbm_params)

[1]	valid_0's multi_logloss: 1.37055
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 1.35591
[3]	valid_0's multi_logloss: 1.34283
[4]	valid_0's multi_logloss: 1.33057
[5]	valid_0's multi_logloss: 1.31957
[6]	valid_0's multi_logloss: 1.30908
[7]	valid_0's multi_logloss: 1.29964
[8]	valid_0's multi_logloss: 1.29079
[9]	valid_0's multi_logloss: 1.28283
[10]	valid_0's multi_logloss: 1.27523
[11]	valid_0's multi_logloss: 1.2685
[12]	valid_0's multi_logloss: 1.2621
[13]	valid_0's multi_logloss: 1.25639
[14]	valid_0's multi_logloss: 1.25115
[15]	valid_0's multi_logloss: 1.24603
[16]	valid_0's multi_logloss: 1.24165
[17]	valid_0's multi_logloss: 1.23721
[18]	valid_0's multi_logloss: 1.23355
[19]	valid_0's multi_logloss: 1.23021
[20]	valid_0's multi_logloss: 1.22685
[21]	valid_0's multi_logloss: 1.22404
[22]	valid_0's multi_logloss: 1.22145
[23]	valid_0's multi_logloss: 1.21919
[24]	valid_0's multi_logloss: 1.21698
[25]	valid_0's multi_logloss: 1.21496

In [10]:
numbers = np.arange(1, 101)
X_test = np.transpose(binary_encode(numbers, NUM_DIGITS))

In [11]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)

In [12]:
y_test = np.array([fizz_buzz_encode(i) for i in range(1, 101)])
y_test

array([0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0,
       1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0,
       0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1,
       2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0,
       1, 0, 2, 1, 0, 0, 1, 2])

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_max)

0.53000000000000003

In [14]:
y_pred

array([[ 0.50956917,  0.26906071,  0.1580794 ,  0.06329073],
       [ 0.51247212,  0.26489663,  0.15897996,  0.06365129],
       [ 0.50956917,  0.26906071,  0.1580794 ,  0.06329073],
       [ 0.5096991 ,  0.27289033,  0.15410365,  0.06330692],
       [ 0.50891651,  0.27859401,  0.14927976,  0.06320972],
       [ 0.50619058,  0.27101189,  0.15992639,  0.06287115],
       [ 0.5031266 ,  0.27542447,  0.15895835,  0.06249059],
       [ 0.49896627,  0.24614118,  0.1435632 ,  0.11132935],
       [ 0.49896627,  0.24614118,  0.1435632 ,  0.11132935],
       [ 0.49991784,  0.24661059,  0.14383699,  0.10963458],
       [ 0.49760532,  0.24546982,  0.14317163,  0.11375323],
       [ 0.49392914,  0.24243511,  0.13895569,  0.12468007],
       [ 0.49125292,  0.24653976,  0.1382028 ,  0.12400453],
       [ 0.49919348,  0.24501901,  0.14043669,  0.11535082],
       [ 0.49880286,  0.25032877,  0.1403268 ,  0.11054157],
       [ 0.51247212,  0.26489663,  0.15897996,  0.06365129],
       [ 0.50956917,  0.

In [15]:
y_pred_max

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
output = np.vectorize(fizz_buzz)(numbers, y_pred_max)
output

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
       '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68',
       '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
       '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90',
       '91', '92', '93', '94', '95', '96', '97', '98', '99', '100'],
      dtype='<U3')

# Weight

In [17]:
rate = []
for i in range(4):
    rate.append(c_train[0]/c_train[i])
    print(rate[i])

1.0
1.9999964230399152
4.0000071539457585
7.999957076632518


1.0
1.9954545454545454
4.027522935779817
7.9818181818181815


7.999957076632518

In [18]:
def weights(n):
    return(rate[n])

w = [weights(i) for i in list(y_train)]

In [19]:
lgbm_params = {
    'boosting_type' : 'gbdt',
    'objective': 'multiclass',
    'num_class': 4,
    'learning_rate': 0.05,
    'min_child_samples': 5,  # Minimum number of data need in a child(min_data_in_leaf)
    'subsample': 0.9,  # Subsample ratio of the training instance.
}

def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df, weight=w)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

In [20]:
model = lgbm_train(X_train, X_valid, y_train, y_valid, lgbm_params)

[1]	valid_0's multi_logloss: 1.38751
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 1.38598
[3]	valid_0's multi_logloss: 1.38706
[4]	valid_0's multi_logloss: 1.38866
[5]	valid_0's multi_logloss: 1.38841
[6]	valid_0's multi_logloss: 1.38913
[7]	valid_0's multi_logloss: 1.39021
[8]	valid_0's multi_logloss: 1.39231
[9]	valid_0's multi_logloss: 1.39463
[10]	valid_0's multi_logloss: 1.39492
[11]	valid_0's multi_logloss: 1.39564
[12]	valid_0's multi_logloss: 1.39674
Early stopping, best iteration is:
[2]	valid_0's multi_logloss: 1.38598


In [21]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)

In [22]:
y_pred

array([[ 0.2512448 ,  0.24984486,  0.25317803,  0.24573231],
       [ 0.2512448 ,  0.24984486,  0.25317803,  0.24573231],
       [ 0.2512448 ,  0.24984486,  0.25317803,  0.24573231],
       [ 0.24541624,  0.25178974,  0.25514885,  0.24764517],
       [ 0.25661042,  0.24805446,  0.25136374,  0.24397138],
       [ 0.24541624,  0.25178974,  0.25514885,  0.24764517],
       [ 0.24867781,  0.25070142,  0.25404601,  0.24657477],
       [ 0.24999052,  0.25033407,  0.24829476,  0.25138065],
       [ 0.24999052,  0.25033407,  0.24829476,  0.25138065],
       [ 0.24999052,  0.25033407,  0.24829476,  0.25138065],
       [ 0.24999052,  0.25033407,  0.24829476,  0.25138065],
       [ 0.24933562,  0.25055266,  0.24851157,  0.25160015],
       [ 0.26064683,  0.24677726,  0.24476693,  0.24780897],
       [ 0.24933562,  0.25055266,  0.24851157,  0.25160015],
       [ 0.25263184,  0.24945246,  0.24742034,  0.25049536],
       [ 0.2512448 ,  0.24984486,  0.25317803,  0.24573231],
       [ 0.2512448 ,  0.

In [23]:
y_pred_max

array([2, 2, 2, 2, 0, 2, 2, 3, 3, 3, 3, 3, 0, 3, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       3, 3, 3, 3, 0, 3, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 3, 0, 3,
       0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 2, 0, 0, 2,
       0, 3, 3, 0, 1, 0, 0, 2, 0, 2, 0, 1, 2, 2, 2, 2, 1, 3, 0, 0, 0, 1, 2,
       2, 2, 2, 0, 2, 2, 2, 0])

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_max)

0.23999999999999999

In [25]:
output = np.vectorize(fizz_buzz)(numbers, y_pred_max)
output

array(['buzz', 'buzz', 'buzz', 'buzz', '5', 'buzz', 'buzz', 'fizzbuzz',
       'fizzbuzz', 'fizzbuzz', 'fizzbuzz', 'fizzbuzz', '13', 'fizzbuzz',
       '15', 'buzz', 'buzz', 'buzz', 'buzz', '20', 'buzz', 'buzz', 'buzz',
       'fizzbuzz', 'fizzbuzz', 'fizzbuzz', 'fizzbuzz', '28', 'fizzbuzz',
       '30', '31', 'buzz', 'buzz', 'buzz', 'buzz', 'buzz', '37', 'buzz',
       'buzz', '40', '41', '42', '43', 'fizzbuzz', '45', 'fizzbuzz', '47',
       'buzz', 'buzz', 'buzz', 'buzz', '52', 'buzz', 'buzz', 'buzz', '56',
       '57', '58', '59', '60', 'fizzbuzz', '62', '63', 'buzz', '65',
       'buzz', '67', '68', 'buzz', '70', 'fizzbuzz', 'fizzbuzz', '73',
       'fizz', '75', '76', 'buzz', '78', 'buzz', '80', 'fizz', 'buzz',
       'buzz', 'buzz', 'buzz', 'fizz', 'fizzbuzz', '88', '89', '90',
       'fizz', 'buzz', 'buzz', 'buzz', 'buzz', '96', 'buzz', 'buzz',
       'buzz', '100'],
      dtype='<U8')

# Sampling

In [26]:
from imblearn.datasets import make_imbalance

X_r, y_r = make_imbalance(X, y,
                      sampling_strategy={0: c_train[3], 1: c_train[3], 2: c_train[3], 3: c_train[3]},
                      random_state=123)

In [27]:
lgbm_params = {
    'boosting_type' : 'gbdt',
    'objective': 'multiclass',
    'num_class': 4,
    'learning_rate': 0.05,
#     'min_child_samples': 5,  # Minimum number of data need in a child(min_data_in_leaf)
    'subsample': 0.9,  # Subsample ratio of the training instance.
}

def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df, weight=None)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

In [28]:
model = lgbm_train(X_r, X_valid, y_r, y_valid, lgbm_params)

[1]	valid_0's multi_logloss: 1.3828
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 1.3806
[3]	valid_0's multi_logloss: 1.37872
[4]	valid_0's multi_logloss: 1.37686
[5]	valid_0's multi_logloss: 1.3756
[6]	valid_0's multi_logloss: 1.37366
[7]	valid_0's multi_logloss: 1.37291
[8]	valid_0's multi_logloss: 1.37212
[9]	valid_0's multi_logloss: 1.37185
[10]	valid_0's multi_logloss: 1.37192
[11]	valid_0's multi_logloss: 1.37205
[12]	valid_0's multi_logloss: 1.37261
[13]	valid_0's multi_logloss: 1.37276
[14]	valid_0's multi_logloss: 1.37344
[15]	valid_0's multi_logloss: 1.37325
[16]	valid_0's multi_logloss: 1.37475
[17]	valid_0's multi_logloss: 1.37444
[18]	valid_0's multi_logloss: 1.37601
[19]	valid_0's multi_logloss: 1.37562
Early stopping, best iteration is:
[9]	valid_0's multi_logloss: 1.37185


In [29]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)

In [30]:
accuracy_score(y_test, y_pred_max)

0.26000000000000001

In [31]:
output = np.vectorize(fizz_buzz)(numbers, y_pred_max)
output

array(['buzz', 'buzz', 'buzz', 'fizzbuzz', 'fizzbuzz', 'fizzbuzz',
       'fizzbuzz', 'fizz', 'fizz', 'fizz', 'fizz', 'fizz', 'fizz', 'fizz',
       'fizz', '16', '17', 'buzz', 'buzz', 'fizzbuzz', 'fizzbuzz',
       'fizzbuzz', 'fizzbuzz', 'fizz', 'fizz', '26', '27', 'fizz', 'fizz',
       'fizz', 'fizz', '32', '33', 'buzz', 'buzz', '36', '37', '38', '39',
       '40', '41', 'fizz', 'fizz', '44', '45', 'fizz', 'fizz', 'buzz',
       'buzz', 'buzz', 'buzz', 'fizzbuzz', 'fizzbuzz', '54', '55', 'fizz',
       'fizz', 'fizz', 'fizz', 'fizz', 'fizz', 'fizz', 'fizz', 'fizzbuzz',
       'fizzbuzz', 'buzz', 'buzz', 'fizz', 'fizz', 'fizz', 'fizz',
       'fizzbuzz', 'fizzbuzz', 'fizzbuzz', 'fizzbuzz', 'fizz', 'fizz',
       'fizz', 'fizz', '80', '81', 'buzz', 'buzz', 'fizz', 'fizz', 'fizz',
       'fizz', '88', '89', '90', '91', 'fizz', 'fizz', 'fizz', 'fizz',
       '96', '97', 'buzz', 'buzz', '100'],
      dtype='<U8')